In [1]:
import requests
from requests.structures import CaseInsensitiveDict
import json
from lxml import etree
import pickle


# 在网易科技频道 network js里找的url
urls = ["https://tech.163.com/special/00097UHL/tech_datalist.js?callback=data_callback", "https://tech.163.com/special/00097UHL/tech_datalist_02.js?callback=data_callback", "https://tech.163.com/special/00097UHL/tech_datalist_03.js?callback=data_callback"]

# 设置请求头伪装
headers = CaseInsensitiveDict()
headers["Referer"] = "https://tech.163.com/"
headers["user-agent"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36"

# 保存访问过的docurl
url_set = set()
news_list = []
cnt = 0

for url in urls:
    resp = requests.get(url, headers=headers)
    if resp.status_code == 200:
        json_data = json.loads(resp.text[len('data_callback('):-1])

        for item in json_data:
            title = item['title']
            docurl = item['docurl']

            if docurl not in url_set:
                
                doc_resp = requests.get(docurl, headers=headers, timeout=10)
                doc_resp.encoding = 'utf-8'
                tree = etree.HTML(doc_resp.text)

                # 定位到新闻正文部分
                post_body = tree.xpath("//div[@class='post_body']")
                # 没有正文的视频。。之类这一步就被筛掉
                if post_body:
                    # 因为p里还有strong之类的。。真费事儿
                    paragraphs = post_body[0].xpath(".//p")
                    html = ''.join(etree.tostring(p, method='html', encoding='unicode') for p in paragraphs)
                    text = etree.HTML(html).xpath("//text()")
                    text = ''.join(t.strip() for t in text if t.strip())
                    
                    news_list.append([docurl, title, text])

                url_set.add(docurl)
            cnt += 1
            if cnt % 15 == 0:
                print(f'{cnt} processed.')


# 保存原文
# wb - 以二进制模式打开文件进行写入 适用于非文本数据（这里因为01序列化）
with open('news_list.dat', 'wb') as file:
    pickle.dump(news_list, file)

15 processed.
30 processed.
45 processed.
60 processed.
75 processed.
90 processed.


In [2]:
import pickle

with open('news_list.dat', 'rb') as file:
    news_list = pickle.load(file)


In [3]:
len(news_list)

89

In [4]:
news_list[20]

['https://www.163.com/tech/article/JCGS7BC200097U7T.html',
 '全新岚图梦想家正式上：搭载华为智驾 32.99万元起',
 '9月20日消息，全新岚图梦想家昨日正式上市并公布价格，其中，全新岚图梦想家四驱卓越鲲鹏版PHEV 32.99万元、EV 34.99万元；全新岚图梦想家四驱尊贵鲲鹏版PHEV 35.99万元、EV 37.99万元；全新岚图梦想家四驱旗舰鲲鹏版PHEV 39.99万元、EV 41.99万元；全新岚图梦想家四驱旗舰乾崑版PHEV 42.99万元、EV 44.99万元。岚图方面表示，在外观方面，全新岚图梦想家以鲲鹏展翅贯穿式前大灯搭配扶摇直上前格栅，配以34根对称华柱，搭配超5.3米的车身。资料显示，全新岚图梦想家车内空间宽敞如7平米大平层，二排配备SPA及零重力座椅，还拥有SPA级10点式按摩与3D环抱式加热功能；新车还配备“大冰箱、大彩电”，拥有13L双开门冷暖智能冰箱，可轻松容纳10瓶矿泉水；17.3英寸二排娱乐屏可随时变身家庭电影院。即便是第三排座椅，乘坐体验也不降级，配有四向电动调节与同级唯一的三排三座加热功能。内饰方面，全新岚图梦想家鲲鹏版也进行了全面焕新，并搭载了岚图自研的逍遥座舱和鲲鹏智能驾驶辅助系统；前排配备双15.4英寸悬浮式联屏，分辨率达2.5K，长途出行时，副驾驶可以尽情K歌、刷剧，且不会影响到驾驶员使用中控屏进行导航。新车不仅为驾驶员提供了仪表屏，还提供了尺寸达29英寸的AR-HUD。全新岚图梦想家逍遥座舱在硬件层面，还提供了冰箱控制屏及二排座椅双扶手屏，为用户带来8屏联动的全新交互体验，车规级高性能8295P芯片的应用算力大幅提升；在软件层面，新车搭载了更聪明的VOYAH GPT大模型，可在500毫秒内极速响应，支持连续对话，识别清晰执行准确且支持全界面可见即可说；鲲鹏智能驾驶辅助系统配备了24个感知智慧传感器与26项智能驾驶辅助功能，车辆智驾水平达到L2.5级。岚图方面表示，全新岚图梦想家依然传承了旗舰级安全、底盘和动力三大硬核实力，为用户带来安心、畅快及舒适的驾乘体验，七重正碰保护与五层追尾保护设计，配合上2000Mpa超高强度笼式车身，获得了C-NCAP五星安全认证，并通过了110km/h时速超美标追尾测试。在底盘方面，全新岚图梦想家采用了前双叉臂、后五连杆独

In [7]:
import re
from IPython.display import HTML, display

# 谷歌红凸显关键词
def highlight(text, keyword):
    return re.sub(pattern=f'({keyword})', repl=r'<span style="color:#dd4b39";>\1</span>', string=text, flags=re.IGNORECASE)

def score(item, keyword):
    keyword_lower = keyword.lower()
    return (item[1].lower().count(keyword_lower) * 5 + 
            item[2].lower().count(keyword_lower) * 3)

keyword = 'iphone'
keyword_lower = keyword.lower()

# list里面每条tuple包括 1 index 2 得分
result = [(i, score(item, keyword)) for i, item in enumerate(news_list) if (keyword_lower in item[1].lower() or keyword_lower in item[2].lower())]
# 根据得分排序
result.sort(key=lambda x: x[1], reverse=True)

# display each result with the title highlighted and clickable
for item in result:
    clickable_title = f'<a href="{news_list[item[0]][0]}" target="_blank">{highlight(news_list[item[0]][1], keyword)}</a>'
    display(HTML(f'[{item[1]}] {clickable_title}'))